In [12]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

In [13]:
analysis_date = dt.date(2022, 1, 12)

In [14]:
samtrans_itp_id = 290

In [116]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [16]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [17]:
from tqdm import tqdm_notebook

In [18]:
from tqdm.notebook import trange, tqdm

In [19]:
pbar = tqdm()

0it [00:00, ?it/s]

In [ ]:
## running about 2 minutes for interpolators, about 

In [21]:
# sam = rt.OperatorDayAnalysis(samtrans_itp_id, analysis_date, pbar)

In [ ]:
(sam.stop_delay_view
 >> group_by(_.stop_id, _.stop_name)
 >> summarize(mean_delay = _.delay.mean(), max_delay = _.delay.max())
 >> arrange(-_.mean_delay)) >> head(3)

In [64]:
sam.set_filter(start_time='15:00', end_time='19:00', direction_id='1')

In [65]:
sam.filter

{'start_time': datetime.time(15, 0),
 'end_time': datetime.time(19, 0),
 'route_ids': None,
 'direction_id': '1',
 'direction': None}

In [66]:
%%capture
m = sam.segment_speed_map(colorscale=colorscale)

In [68]:
# m

### speedup ideas

* selective use of numba @jit decorator in computation-heavy methods?
    * likely requires refactor to plain np, translation, etc...

In [117]:
rt.TripPositionInterpolator.detailed_speed_map(ex1)

/home/jovyan/data-analyses/rt_delay/rt_analysis.py:109: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

